# Testing Hypothesis

가설 검증 

In [2]:
# 경고 메시지 출력 끄기
import warnings 
warnings.filterwarnings(action='ignore')

# 노트북 셀 표시를 브라우저 전체 폭 사용하기
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import clear_output

%matplotlib inline
import matplotlib.pyplot as plt

import os, sys, shutil, functools
import collections, pathlib, re, string

rseed = 22
import random
random.seed(rseed)

import numpy as np
np.random.seed(rseed)
np.set_printoptions(precision=5)
np.set_printoptions(formatter={'float_kind': "{:.5f}".format})

import pandas as pd
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.5f}'.format

import scipy as sp

import seaborn as sns

from pydataset import data

print(f"python ver={sys.version}")
print(f"pandas ver={pd.__version__}")
print(f"numpy ver={np.__version__}")
print(f"scipy ver={sp.__version__}")

python ver=3.8.9 (default, Jun 27 2021, 02:41:12) 
[GCC 7.5.0]
pandas ver=1.2.5
numpy ver=1.19.5
scipy ver=1.6.3


In [3]:
# Iris 데이터 셋의 컬럼 정보 살피기
df_iris = data('iris')
df_iris.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 1 to 150
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  150 non-null    float64
 1   Sepal.Width   150 non-null    float64
 2   Petal.Length  150 non-null    float64
 3   Petal.Width   150 non-null    float64
 4   Species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 7.0+ KB


## 가설검정 (Testing Hypothesis)

가설검정이란 모집단의 통계량에 대해 두가지 가설을 세우고 표본의 통계량을 이용하여 어느 가설이 옳은지 판단하는 통계적 기법입니다. 모집단 통계량에 대해 귀무가설(H0, Null hypothesis, 기각하고 싶은 가설)을 세우고 이에 대한 표본을 수집하여 계산한 표본통계량(검정통계량)이 확률적으로 거의 일어나지 않을 일임을 증명하여 귀무가설을 기각하고, 대립가설(H1, Alternative hypothesis, 채택하고 싶은 가설)을 채택하는 방식입니다. 이를 위해 중심극한정리와 구간추정이 사용됩니다.

먼저 가설검정에 사용되는 용어를 정리합니다.

* 귀무가설(H0, Null hypothesis): 기각하고 싶은 가설
* 대립가설(H1, Alternative hypothesis): 채택하고 싶은 가설
* 기각역(Rejection region): 귀무가설이 기각되는 구간
* 채택역(Acceptance region): 귀무가설이 채택되는 구간
* 유의수준(Significance level): 기각역 구간에 들어가는 확률
* 임계값(Critical value): 기각역 구간의 경계선 상의 값
* 검정통계량(Test statistic): 검정에 사용되는 통계량
* p값(p-value): 검정통계량의 확률 값

이제 가설 검증의 단계에 대해서 알아봅니다.

**가설검증 단계**

![](https://storage.ning.com/topology/rest/1.0/file/get/4345192898?profile=original)

예) 모평균과 표본평균 검정

매일 사먹는 감자칩에 표기된 무게 150g인데 사먹을 때마다 왠지 부족하다는 생각이 들어 검증을 하고 싶어졌다.

* Step1: 귀무가설 정의
  * H0: $\mu = \mu_0$, 감자칩의 평균 무게가 150g 이다.
* Step2: 대립가설 정의
  * H1: $\mu \neq \mu_0$, 감자칩의 평균 무게가 150g이 아니다.

귀무가설과 대립가설의 정의에 따라 양측검정(Two-tailed Test), 단측검정(One-tailed Test) 실시 여부가 결정 됩니다.

![](https://ars.els-cdn.com/content/image/3-s2.0-B9780128008522000092-f09-06-9780128008522.jpg)

* Step3: 유의수준 정의
  * 유의 수준: 0.05, 귀무가설 기각을 위한 기각역의 확률   
* Step4: 검증 데이터 선택
  * 랜덤하게 5개의 감자칩을 구매하여 무게 측정
* Step5: 검정 통계량 또는 p-value 계산
  * 보통 모분산을 알수 없고, 소표본에서도 사용할 수 있는 t-score 를 이용하여 검정 통계량을 계산 또는 p-value 계산 (One Sample T-Test)

$$t = \frac{\bar{x} - \mu}{s/\sqrt{n}}$$

![](http://scu2.anu.edu.au/core/HtestMean/images/twoTailedT.gif)
* Step6: 임계값 계산 
  * 자유도가 4인 t 분포에서 임계값 계산

In [60]:
# 표본(Sample): 5개의 데이터를 가지는 표본을 랜덤 샘플링
samples = [134, 125, 150, 145, 134]
size = len(samples)

# 신뢰구간 (confidence level), 유의수준 (significance level), 자유도 (Degree of Freedom)
cl = .95
sl = (1.-cl)/2 # for Two-Tailed (양측검정) 
dof = size - 1 # Degree of Freedom
ddof = 1 # Delta Degrees of Freedom (dof = n - ddof)
print(f"cl={cl}, sl={sl:.3f}, dof={dof}, ddof={ddof}")

# 표본평균(Sample Mean), 표본편차(Sample Standard Deviation) , 표준오차 (Standard Error)
s_mean = np.mean(samples)
s_std = np.std(samples) 
s_ste = s_std / np.sqrt(size)
t_score = (s_mean - 150)/s_ste
print(f"t_score={t_score:.5f}")

# c_value: Critical value계산, interval 함수 이용 
c_value = sp.stats.t.interval(alpha=cl, df=dof) 
print(f"c_value:{c_value[0]:.5f}, {c_value[1]:.5f}")

# p-value: 양측 검정이기 때문에 면적x2
p_value = sp.stats.t.cdf(t_score, dof)*2
# p_value = sp.stats.t.sf(abs(t_score), dof)*2
print(f"p_value={p_value:.5f}")

cl=0.95, sl=0.025, dof=4, ddof=1
t_score=-3.12669
c_value:-2.77645, 2.77645
p_value=0.03530


* Step7: 귀무가설 채택/기각 결정
  * 검정 통계량이 임계값보다 작거나 크면 기각역이므로 귀무가설을 기각하고 대립가설을 채택합니다.
  * p-value 가 유의수준보다 작으면 기각역이므로 귀무가설을 기각하고 대립가설을 채택합니다.

검정 통계량인 표본평균의 t-score 가 t분포의 유의수준 임계값 범위를 벗어나기 때문에 귀무가설을 기각하고 대립가설을 채택합니다.

t-score의 p-value 값이 유의수준인 0.05 보다 작아 귀무가설을 기가하고 대립가설을 채택합니다.